# The MNIST Data

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Implementing the Regression

In [2]:
import tensorflow as tf
x = tf.placeholder(tf.float32, [None, 784])

We want to be able to input any number of MNIST images, each flattened into a 784-dimensional vector. We represent this as a 2-D tensor of floating-point numbers, with a shape ```[None, 784]```. (Here None means that a dimension can be of any length.)

In [3]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Notice that ```W``` has a shape of [784, 10] because we want to multiply the 784-dimensional image vectors by it to produce 10-dimensional vectors of evidence for the difference classes. ```b``` has a shape of [10] so we can add it to the output.

In [4]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

This is how we implement our model. First, we multiply ```x``` by ```W``` with the expression ```tf.matmul(x, W)```. This is flipped from when we multiplied them in our equation, where we had Wx, as a small trick to deal with ```x``` being a 2D tensor with multiple inputs. We then add ```b```, and finally apply ```tf.nn.softmax```.

# Training

Add a new placeholder to input the correct answers.

In [5]:
y_ = tf.placeholder(tf.float32, [None, 10])

Implement the cross-entropy function.

In [6]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [7]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In this case, we ask TensorFlow to minimize cross_entropy using the gradient descent algorithm with a learning rate of 0.5.

We can now launch the model in an ```InteractiveSession```:

In [8]:
sess = tf.InteractiveSession()

We first have to create an operation to initialize the variables we created:

In [9]:
tf.global_variables_initializer().run()

Let's train -- we'll run the training step 1000 times!

In [16]:
import time
start_time = time.time()
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
elapsed_time = time.time() - start_time
print(elapsed_time)

1.2238202095031738


# Evaluating Our Model

In [11]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

That gives us a list of booleans. To determine what fraction are correct, we cast to floating point numbers and then take the mean.

In [12]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9175
